# Download required Python modules

In [20]:
!pip install pandas==2.2.2
!pip install numpy==2.0.0
!pip install mxnet==1.9.1
!pip install --upgrade pandas

  Using cached numpy-2.0.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (60 kB)
Using cached numpy-2.0.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (19.3 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.3
    Uninstalling numpy-1.24.3:
      Successfully uninstalled numpy-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
accelerate 0.32.1 requires numpy<2.0.0,>=1.17, but you have numpy 2.0.0 which is incompatible.
albucore 0.0.12 requires numpy<2,>=1.24.4, but you have numpy 2.0.0 which is incompatible.
arviz 0.18.0 requires numpy<2.0,>=1.23.0, but you have numpy 2.0.0 which is incompatible.
cudf-cu12 24.4.1 requires numpy<2.0a0,>=1.23, but you have numpy 2.0.0 which is incompatible.
cudf-cu12 24.4.1 requires pandas<2.2.2dev0,>=2.0, but you have pandas 2.2.2 which is incompatible.
cupy-cud

In [21]:
!pip install gluonts
!pip install --upgrade mxnet==1.6.0
!pip install "gluonts[torch]"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 22.6 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.0
    Uninstalling numpy-2.0.0:
      Successfully uninstalled numpy-2.0.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pandas<2.2.2dev0,>=2.0, but you have pandas 2.2.2 which is incompatible.


# List all Python modules

In [36]:
!pip list

Package                          Version
-------------------------------- ---------------------
absl-py                          1.4.0
accelerate                       0.32.1
aiohttp                          3.9.5
aiosignal                        1.3.1
alabaster                        0.7.16
albucore                         0.0.12
albumentations                   1.4.11
altair                           4.2.2
annotated-types                  0.7.0
anyio                            3.7.1
argon2-cffi                      23.1.0
argon2-cffi-bindings             21.2.0
array_record                     0.5.1
arviz                            0.18.0
asn1crypto                       1.5.1
astropy                          6.1.2
astropy-iers-data                0.2024.7.22.0.34.13
astunparse                       1.6.3
async-timeout                    4.0.3
atpublic                         4.1.0
attrs                            23.2.0
audioread                        3.0.1
autograd                

### Import required Python **modules**

In [23]:
import numpy as np
import pandas as pd
import os
import matplotlib as mpl
import matplotlib.pyplot as plt
from gluonts.torch.model.deepar import DeepAREstimator
from gluonts.dataset.common import ListDataset
from gluonts.dataset.field_names import FieldName
from gluonts.evaluation.backtest import make_evaluation_predictions
from tqdm.autonotebook import tqdm
from gluonts.evaluation import Evaluator
from typing import Dict

### Define code

In [24]:
# https://www.kaggle.com/datasets/robikscube/hourly-energy-consumption/data?select=COMED_hourly.csv
##df_comed = pd.read_csv("/content/COMED_hourly.csv", parse_dates=True)
# CUSTOM: Correct links
df_comed                    = pd.read_csv("https://media.geeksforgeeks.org/wp-content/uploads/20231221132006/COMED_hourly.csv", parse_dates=True)

# https://www.kaggle.com/datasets/robikscube/hourly-energy-consumption/data?select=DOM_hourly.csv
##df_dom = pd.read_csv("/content/DOM_hourly.csv", parse_dates=True)
# CUSTOM: Correct links
df_dom                      = pd.read_csv("https://drive.usercontent.google.com/uc?id=1fAyiCpja5mWd4CCeMC0sWLylfuFi5Ee6&export=download", parse_dates=True)

# DEBUG
##print( f"df_comed:  { df_comed }" )
##print( f"df_dom:    { df_dom }" )

# Slicing the datasets to make equal lengh data
df_comed                    = df_comed.loc[df_comed["Datetime"] > '2011-12-31'].reset_index(drop=True)
df_dom                      = df_dom.loc[df_dom["Datetime"] > '2011-12-31'].reset_index(drop=True)
df_comed                    = df_comed.T
df_comed.columns            = df_comed.iloc[0]
df_comed                    = df_comed.drop(df_comed.index[0])
df_comed['Station_Name']    = "COMED"
df_comed                    = df_comed.reset_index(drop=True)
df_dom                      = df_dom.T
df_dom.columns              = df_dom.iloc[0]
df_dom                      = df_dom.drop(df_dom.index[0])
df_dom['Station_Name']      = "DOM"
df_dom                      = df_dom.reset_index(drop=True)
df_all                      = pd.concat([df_comed, df_dom], axis=0)
df_all                      = df_all.set_index("Station_Name")
df_all                      = df_all.reset_index()

ts_code                     = df_all['Station_Name'].astype('category').cat.codes.values

freq = "1H"  # rate at  which dataset is sampled

##start_train               = pd.Timestamp("2011-12-31 01:00:00", freq=freq)        # ERROR: Deprecated
start_train                 = pd.Timestamp("2011-12-31 01:00:00").to_period(freq)   # FIX

##start_test = pd.Timestamp("2016-06-10 18:00:00", freq=freq)                       # ERROR: Deprecated
start_test = pd.Timestamp("2016-06-10 18:00:00").to_period(freq)                    # FIX

prediction_lentgh = 24 * 1  # Our prediction Length is 1 Day

# Dataset split: Training and testing sets split
df_train = df_all.iloc[:, 1:40000].values
df_test = df_all.iloc[:, 40000:].values

### Model: DeepAR estimator
Hyperparameters:

---
- **freq**: this parameter defines the frequency of the time series data. It represents the number of time steps in one period or cycle of the time series. Our data has daily observations, so, the frequency is determined by the variable freq.

- **context_length**: This parameter sets the number of time steps that the model uses to learn patterns and dependencies in the historical data. Here it is set to (24 * 5), indicating that the model looks back over a period equivalent to 5 days (by assuming each time step corresponds to an hour).

- **prediction_length**: This parameter specifies how far into the future the model should generate predictions. It determines the length of the forecast horizon.

- **cardinality**: This parameter is a list that indicates the number of categories for each categorical feature in the dataset.

- **num_layers**: It determines the number of layers in the neural network architecture. In our case, the model is configured with 2 layers.

- **dropout_rate**: It is a regularization technique that helps prevent overfitting. It represents the fraction of input units to drop out during training. A value of 0.25 means that 25% of the input units will be randomly set to zero during each update.

- **trainer_kwargs**: This is a dictionary containing additional arguments for the training process. In our case, it includes 'max_epochs': 16, which sets the maximum number of training epochs. An epoch is one complete pass through the entire training dataset.

In [25]:
# DEBUG
print( df_train)
print("")
print( df_test)

CONFIG_NEURAL_NETWORK_EPOCHS                    = 16

estimator = DeepAREstimator(freq                = freq,
                            context_length      = 24 * 5, # context length is number of time steps to look back(5 days in a week)
                            prediction_length   = prediction_lentgh,
                            cardinality         = [1],
                            num_layers          = 2,
                            dropout_rate        = 0.25,
                            trainer_kwargs      = {'max_epochs': CONFIG_NEURAL_NETWORK_EPOCHS}
                            )

[[9970.0 9428.0 9059.0 ... 16409.0 16977.0 17412.0]
 [8547.0 8197.0 7932.0 ... 12560.0 12960.0 13351.0]]

[[17744.0 17790.0 17437.0 ... 13627.0 13336.0 12816.0]
 [13587.0 13383.0 12743.0 ... 18307.0 17814.0 17428.0]]


### Dataset preparation
For every deep learning model, it is required to prepare the raw dataset as per model’s input type. Here also, we need to redefine our raw dataset for training and testing.

In [26]:
train_ds = ListDataset([
    {
        FieldName.TARGET:           target,
        FieldName.START:            start_train,
        FieldName.FEAT_STATIC_CAT:  fsc
    }
    for (target, fsc) in zip(df_train,
                             ts_code.reshape(-1, 1))
], freq=freq)

test_ds = ListDataset([
    {
        FieldName.TARGET:           target,
        FieldName.START:            start_test,
        FieldName.FEAT_STATIC_CAT:  fsc
    }
    for (target, fsc) in zip(df_test,
                             ts_code.reshape(-1, 1))
], freq=freq)

# DEBUG
print( train_ds)
print("")
print( test_ds)

[{'target': array([ 9970.,  9428.,  9059., ..., 16409., 16977., 17412.], dtype=float32), 'start': Period('2011-12-31 01:00', 'H'), 'feat_static_cat': array([0], dtype=int32)}, {'target': array([ 8547.,  8197.,  7932., ..., 12560., 12960., 13351.], dtype=float32), 'start': Period('2011-12-31 01:00', 'H'), 'feat_static_cat': array([1], dtype=int32)}]

[{'target': array([17744., 17790., 17437., ..., 13627., 13336., 12816.], dtype=float32), 'start': Period('2016-06-10 18:00', 'H'), 'feat_static_cat': array([0], dtype=int32)}, {'target': array([13587., 13383., 12743., ..., 18307., 17814., 17428.], dtype=float32), 'start': Period('2016-06-10 18:00', 'H'), 'feat_static_cat': array([1], dtype=int32)}]


###Model Training
Now we are ready to train the DeepAR estimator by passing the training dataset which is just prepared. Here we will utilize four CPU code as workers for fast processing.

In [27]:
CONFIG_NUMBER_WORKERS = 4

predictor = estimator.train( training_data=train_ds, num_workers = CONFIG_NUMBER_WORKERS )

INFO: GPU available: False, used: False
INFO:lightning.pytorch.utilities.rank_zero:GPU available: False, used: False
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/local/lib/python3.10/dist-packages/lightning/pytorch/trainer/configuration_validator.py:74: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
INFO: 
  | Name  | Type        | Params | In sizes                                                      | Out sizes   
---------------------------------------------------------------------------------------------------------------------
0 | model | DeepARModel | 27.6 K | [[1, 1], [1, 1], [1, 840, 5], [1, 840], [1, 840], [1, 24, 5]

Training: |          | 0/? [00:00<?, ?it/s]

INFO: Epoch 0, global step 50: 'train_loss' reached 9.75728 (best 9.75728), saving model to '/content/lightning_logs/version_2/checkpoints/epoch=0-step=50.ckpt' as top 1
INFO:lightning.pytorch.utilities.rank_zero:Epoch 0, global step 50: 'train_loss' reached 9.75728 (best 9.75728), saving model to '/content/lightning_logs/version_2/checkpoints/epoch=0-step=50.ckpt' as top 1
INFO: Epoch 1, global step 100: 'train_loss' reached 8.79226 (best 8.79226), saving model to '/content/lightning_logs/version_2/checkpoints/epoch=1-step=100.ckpt' as top 1
INFO:lightning.pytorch.utilities.rank_zero:Epoch 1, global step 100: 'train_loss' reached 8.79226 (best 8.79226), saving model to '/content/lightning_logs/version_2/checkpoints/epoch=1-step=100.ckpt' as top 1
INFO: Epoch 2, global step 150: 'train_loss' reached 8.39175 (best 8.39175), saving model to '/content/lightning_logs/version_2/checkpoints/epoch=2-step=150.ckpt' as top 1
INFO:lightning.pytorch.utilities.rank_zero:Epoch 2, global step 150: '

### Forecasting
Plot forecasted values versus actual ones

In [53]:
forecast_it, ts_it = make_evaluation_predictions(   dataset     = test_ds,
                                                    predictor   = predictor,
                                                    num_samples = 100, )

print("Gathering time series conditioning values ...")
tss = list( tqdm( ts_it, total = len( df_test ) ) )

print("Gathering time series predictions ...")
forecasts = list( tqdm( forecast_it, total = len( df_test ) ) )

def plot_prob_forecasts(ts_entry, forecast_entry):
    plot_length = prediction_lentgh
    prediction_intervals = (0.5, 0.8)
    legend = ["observations", "median prediction"] + \
        [f"{k*100}% prediction interval" for k in prediction_intervals][::-1]

    fig, ax = plt.subplots(1, 1, figsize=(10, 7))
    ts_entry[-plot_length:].plot(ax=ax, color='blue', label='observations')

    # Extract the median prediction
    median = np.median(forecast_entry, axis=0)
    ax.plot(ts_entry.index[-plot_length:], median[-plot_length:],
            color='orange', label='median prediction')

    # Extract the prediction intervals if available
    if len(forecast_entry) > 1:
        lower, upper = np.percentile(forecast_entry, q=[(1 - k) * 100 / 2 for k in prediction_intervals], axis=0), np.percentile(
            forecast_entry, q=[(1 + k) * 100 / 2 for k in prediction_intervals], axis=0)

        # Ensure lower and upper are 1-D arrays
        lower, upper = lower[-plot_length:], upper[-plot_length:]

    plt.grid(which="both")
    plt.legend(legend, loc="upper left")
    plt.show()


for i in tqdm(range(2)):
    ts_entry = tss[i]
    forecast_entry = np.array(forecasts[i].samples)
    plot_prob_forecasts(ts_entry, forecast_entry)

NameError: name 'matplotlib' is not defined

### Model evaluation
Evaluate the trained model

In [ ]:
evaluator = Evaluator(quantiles=[0.1, 0.5, 0.9])

agg_metrics, item_metrics = evaluator(iter(tss), iter(forecasts), num_series=len(df_test))

item_metrics